In [11]:
import json
import pickle
import numpy as np
import nltk
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense,Dropout
from tensorflow.keras.optimizers import SGD

ignore_words=["?","¿","!","¡"]
data_file=open("intents.json")
intents= json.load(data_file)

words=[]
classes=[]
documents=[]

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        w= nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w,intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])


from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')
words = [stemmer.stem(w.lower())for w in words if w not in ignore_words]
pickle.dump(words,open("words.pkl","wb"))
pickle.dump(classes,open("classes.pkl","wb"))

training =[]
output_empty=[0]*len(classes)

for doc in documents:
    bag=[]
    pattern_words=doc[0]
    pattern_words=[stemmer.stem(word.lower())for word in pattern_words if word not in ignore_words]
    for palabra in words:
        bag.append(1) if palabra in pattern_words else bag.append(0)
    output_row= list(output_empty)
    output_row[classes.index(doc[1])]=1
    training.append([bag,output_row])

training=np.array(training)
x_train= list(training[:,0])
y_train= list(training[:,1])

model = Sequential()
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]),activation='softmax'))

sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)

model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=['accuracy'])

hist=model.fit(np.array(x_train),np.array(y_train),epochs=300,batch_size=5,verbose=1)
model.save("chatbot_model.h5",hist)
print("modelo creado")



C:\Users\ck\AppData\Local\Temp\ipykernel_3512\1895264844.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)
c:\Users\ck\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/300
6/6 [==============================] - 6s 8ms/step - loss: 1.3801 - accuracy: 0.2593
Epoch 2/300
6/6 [==============================] - 0s 7ms/step - loss: 1.2600 - accuracy: 0.4444
Epoch 3/300
6/6 [==============================] - 0s 8ms/step - loss: 1.1485 - accuracy: 0.5185
Epoch 4/300
6/6 [==============================] - 0s 8ms/step - loss: 1.0397 - accuracy: 0.5926
Epoch 5/300
6/6 [==============================] - 0s 7ms/step - loss: 1.0462 - accuracy: 0.4815
Epoch 6/300
6/6 [==============================] - 0s 10ms/step - loss: 0.9520 - accuracy: 0.6296
Epoch 7/300
6/6 [==============================] - 0s 9ms/step - loss: 0.8324 - accuracy: 0.5926
Epoch 8/300
6/6 [==============================] - 0s 8ms/step - loss: 0.7949 - accuracy: 0.7037
Epoch 9/300
6/6 [==============================] - 0s 7ms/step - loss: 0.7262 - accuracy: 0.7037
Epoch 10/300
6/6 [==============================] - 0s 6ms/step - loss: 0.6720 - accuracy: 0.6667
Epoch 11/300
6/6 [==========

In [12]:
import nltk, json,pickle
import numpy as np

from tensorflow.keras.models import load_model
stemmer = SnowballStemmer('spanish')


model=load_model("chatbot_model.h5")
intents= json.loads(open("intents.json").read())
words=pickle.load(open("words.pkl","rb"))
classes=pickle.load(open("classes.pkl","rb"))

def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence) 
    sentence_words=[stemmer.stem(word.lower()) for word in sentence_words] 
    return sentence_words


def bow (sentence,words,show_details=True): 
    sentence_words=clean_up_sentence(sentence)
    
    bag=[0]*len(words)
    
    for i in sentence_words:
        for j,w in enumerate(words):
            if w==i: 
                bag[j]=1
                if show_details:
                    print("encontrado en la bolsa: ",w)
    return (np.array(bag))

def predict_class(sentence,model):
    p = bow(sentence,words,show_details=False)
    res = model.predict(np.array([p]))[0] 
    
    ERROR_THRESHOLD=0.25 
    results= [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD] 

    results.sort(key=lambda x: x[1], reverse=True)

    return_list[]
    
    for r in results:
        return_list[].append({"intent":classes[r[0]],"probability":str(r[1])})
    
    return return_list

SyntaxError: invalid syntax (1205321454.py, line 41)